<a href="https://colab.research.google.com/github/dhyannn/LLM/blob/main/Dhyan_514_LLMlab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

# Load your dataset
df = pd.read_csv("/content/DataSet_Misinfo_FAKE.csv")
texts = df['text'].tolist()

In [13]:
df.head()


,Unnamed: 0,text
0,0,Donald Trump just couldn t wish all Americans ...
1,1,House Intelligence Committee Chairman Devin Nu...
2,2,"On Friday, it was revealed that former Milwauk..."
3,3,"On Christmas day, Donald Trump announced that ..."
4,4,Pope Francis used his annual Christmas Day mes...


In [4]:
from transformers import RobertaTokenizer

# Load a pre-trained tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [5]:
# Tokenize and encode the dataset
encoded_inputs = tokenizer(
    texts,                       # The list of texts to tokenize
    padding=True,                # Pad to the longest sequence in the batch
    truncation=True,             # Truncate sequences to the maximum length the model can handle
    max_length=512,              # Maximum length of the tokenized sequence
    return_tensors="pt"          # Return PyTorch tensors
)

# Extract input_ids and attention masks
input_ids = encoded_inputs["input_ids"]
attention_masks = encoded_inputs["attention_mask"]


In [14]:
import torch
from torch.utils.data import Dataset

# Create a custom Dataset class
class TextDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings.input_ids)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

if 'text' in df.columns:
    labels = df['text'].tolist()
else:
    labels = None  # Set to None if you don't have labels

# Create the dataset
dataset = TextDataset(encoded_inputs, labels)


In [15]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments

# Load a pre-trained model for sequence classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=len(set(labels)))

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)

# Start training
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-14-56e14c045503>:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TypeError: new(): invalid data type 'str'

In [ ]:
tokenizer.save_pretrained("./financial_tokenizer")
model.save_pretrained("./financial_model")


In [ ]:
from transformers import RobertaForSequenceClassification, Trainer, TrainingArguments
import torch

labels = torch.tensor(df['text'].tolist())

# Incorporate labels into a dictionary
train_dataset = {
    'input_ids': encoded_inputs['input_ids'],
    'attention_mask': encoded_inputs['attention_mask'],
    'text': labels
}

# Load a pre-trained model for sequence classification
model = RobertaForSequenceClassification.from_pretrained("roberta-base")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Create a Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset # Pass the dictionary with labels
)

# Start training
trainer.train()